In [ ]:
import numpy as np
import torch 
import os
from collections import Counter, OrderedDict
import pandas as pd
from IPython.display import display, HTML
import numbers


In [ ]:
'''
Load results of all methods tested
'''

base_folders = './experiments/domain2/'

raw_records = {}
seed_folders = ['1111/'] # select the seed that you want to check

print('Methods to check')
for seed_folder in seed_folders:
    
    raw_records[seed_folder] = {}
    for exp_folder in sorted(os.listdir(base_folders + seed_folder)):
    
        if exp_folder != '.DS_Store':
            print (exp_folder)
            path = base_folders + seed_folder + exp_folder + '/lm/model/' # replace to '/cvae/modeel/' if you test cvae
            raw_records[seed_folder][exp_folder] = []
            for file in sorted(os.listdir(path)):

                if file != '.DS_Store':
                    ckp = torch.load(path+file,map_location='cpu')['perf']
                    raw_records[seed_folder][exp_folder].append(ckp)

In [ ]:
def tabulate_results_all(records):
    '''
    tabulate results for omega_all, omega_first
    '''
    
    formated_results = []
    all_results = {}
    for seed, results in records.items():
        for method, perf in results.items():

            result = []
            sequence = [0,5,2,1,3,4]
            for run in perf:
                # all
                result.append(run['cul_se'])
                result.append(run['cul_bleu'][3])
                # base (the first domain: 0) replace to the first of {task_seq} in c{onfig} if you use other order
                first = 0
                result.append(run['se'][first])
                result.append(run['bleu'][first][3])              

            result = np.asarray(result).reshape(6,4).T.mean(1)
            
            if method in all_results.keys():
                all_results[method] += [result]
            else:
                all_results[method] = [result]
    
    for method, result in all_results.items():
        result = np.array(result)
        mean_result = list(result.mean(axis=0))
    
        formated_results.append( [method] + mean_result )

    headers = ['Methods', 'ALL SER', 'ALL BLEU', 'First SER','First BLEU']
    
    return formated_results, headers

In [ ]:
results, headers = tabulate_results_all(raw_records)
dataframe = pd.DataFrame(results, columns=headers)
display(HTML(dataframe.to_html()))